In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime, timedelta, date

In [3]:
main_df = pd.read_csv('../data/requests_census_neighborhoods.csv')
main_df

/var/folders/46/zm0dtt4s22d760_6ml4_t_vr0000gn/T/ipykernel_54695/1183661926.py:1: DtypeWarning: Columns (2,3,5,7,8,9,10,47) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv('../data/requests_census_neighborhoods.csv')


,ZIP,Households (Count),White (Count),Black or African American (Count),American Indian and Alaska Native (Count),Asian (Count),Native Hawaiian and Other Pacific Islander (Count),Some other race (Count),Two or more races (Count),Hispanic or Latino origin (of any race) (Count),...,Address,City,Council District,Latitude,Longitude,Mapped Location,Time to Resolve,Hours to Resolve,Days to Resolve,Neighborhood
0,37013,"35,483","16,986","14,101",90.0,"1,413",0.0,"1,235","1,658","3,971",...,3613 Wells Ct,ANTIOCH,33.0,36.080968,-86.602406,"(36.080968, -86.602406)",4 days 23:09:25,119.156944,4.964873,Antioch / Cane Ridge
1,37013,"35,483","16,986","14,101",90.0,"1,413",0.0,"1,235","1,658","3,971",...,2371 Una Antioch Pike,ANTIOCH,28.0,36.072191,-86.664342,"(36.07219072, -86.66434192)",2 days 16:16:00,64.266667,2.677778,Antioch / Cane Ridge
2,37013,"35,483","16,986","14,101",90.0,"1,413",0.0,"1,235","1,658","3,971",...,109 Grovedale Trace,ANTIOCH,33.0,36.050526,-86.606362,"(36.05052579, -86.60636223)",35 days 15:25:00,855.416667,35.642361,Antioch / Cane Ridge
3,37013,"35,483","16,986","14,101",90.0,"1,413",0.0,"1,235","1,658","3,971",...,3005 W Towne Village Ct,ANTIOCH,29.0,36.073930,-86.609670,"(36.07393, -86.60967)",7 days 00:04:00,168.066667,7.002778,Antioch / Cane Ridge
4,37013,"35,483","16,986","14,101",90.0,"1,413",0.0,"1,235","1,658","3,971",...,5045 Mountain Springs Rd,ANTIOCH,32.0,36.044549,-86.612186,"(36.0445494, -86.6121858)",1 days 04:29:00,28.483333,1.186806,Antioch / Cane Ridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722021,37228,"1,249",496,533,0.0,16,33.0,0,171,0,...,1927 5th Ave N,NASHVILLE,21.0,36.187059,-86.797371,"(36.187058700238445, -86.7973711900413)",0 days 00:00:00,0.000000,0.000000,Metrocenter
722022,37228,"1,249",496,533,0.0,16,33.0,0,171,0,...,Great Circle Rd & Rosa L Parks Blvd,NASHVILLE,2.0,36.193622,-86.806912,"(36.19362231161992, -86.80691239999999)",NaN,NaN,NaN,Metrocenter
722023,37228,"1,249",496,533,0.0,16,33.0,0,171,0,...,2014 Rosa L Parks Blvd,NASHVILLE,2.0,36.187579,-86.800050,"(36.1875790690887, -86.8000497110188)",67 days 05:44:04,1613.734444,67.238935,Metrocenter
722024,37228,"1,249",496,533,0.0,16,33.0,0,171,0,...,101 Athens Way,NASHVILLE,2.0,36.198301,-86.798659,"(36.1983009, -86.7986588)",0 days 00:00:00,0.000000,0.000000,Metrocenter
